HAND RECOGNITION AND LANDMARKS ESTIMATION 

In [ ]:
from cvzone.HandTrackingModule import HandDetector
from cvzone import putTextRect
from cvzone.FPS import FPS 
import cv2

# Initialize the webcam to capture video
# The '2' indicates the third camera connected to your computer; '0' would usually refer to the built-in camera
cap = cv2.VideoCapture(0)
fpsReader = FPS(avgCount=30)

# Initialize the HandDetector class with the given parameters
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.5, minTrackCon=0.5)

# Continuously get frames from the webcam
while True:
    # Capture each frame from the webcam
    # 'success' will be True if the frame is successfully captured, 'img' will contain the frame
    success, img = cap.read()

    # Find hands in the current frame
    # The 'draw' parameter draws landmarks and hand outlines on the image if set to True
    # The 'flipType' parameter flips the image, making it easier for some detections
    hands, img = detector.findHands(img, draw=True, flipType=True)

    # Check if any hands are detected
    if hands:
        # Information for the first hand detected
        hand1 = hands[0]  # Get the first hand detected
        lmList1 = hand1["lmList"]  # List of 21 landmarks for the first hand
        bbox1 = hand1["bbox"]  # Bounding box around the first hand (x,y,w,h coordinates)
        center1 = hand1['center']  # Center coordinates of the first hand
        handType1 = hand1["type"]  # Type of the first hand ("Left" or "Right")

        # Count the number of fingers up for the first hand
        fingers1 = detector.fingersUp(hand1)
        print(fingers1)

        #print(f'H1 = {fingers1.count(1)}', end=" ")  # Print the count of fingers that are up

        # if handType1=='Right' and fingers1[4] and fingers1.count(1)==1: 
        #     #Take off command
   

        # Check if a second hand is detected
        if len(hands) == 2:
            # Information for the second hand
            hand2 = hands[1]
            lmList2 = hand2["lmList"]
            bbox2 = hand2["bbox"]
            center2 = hand2['center']
            handType2 = hand2["type"]

            # Count the number of fingers up for the second hand
            fingers2 = detector.fingersUp(hand2)
            print(f'H2 = {fingers2.count(1)}', end=" ")

        print(" ") #Print this for a new line


    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(10, 470),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    # Display the image in a window
    cv2.imshow("Drone Simulation", img)

    # Keep the window open and update it for each frame; wait for 1 millisecond between frames
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

DRONE HEALTH PARAMETERS

In [ ]:
from dronekit import connect, VehicleMode, LocationGlobalRelative
import time 

# Connect to the Vehicle (use the correct SITL connection string)
print("Connecting to vehicle on tcp:127.0.0.1:5763 and udp 127.0.0.1:14550")
#vehicle = connect('tcp:127.0.0.1:5762', wait_ready=True)
vehicle = connect('127.0.0.1:14550', wait_ready=True)

# Print some basic info
print("=== Vehicle Info ===")
print(f" Autopilot Firmware version: {vehicle.version}")
print(f" Global Location: {vehicle.location.global_frame}")
print(f" Global Location (relative altitude): {vehicle.location.global_relative_frame}")
print(f" Local Location: {vehicle.location.local_frame}")  # NED
print(f" Attitude: {vehicle.attitude}")
print(f" Velocity: {vehicle.velocity}")
print(f" GPS: {vehicle.gps_0}")
print(f" Battery: {vehicle.battery}")
print(f" EKF OK?: {vehicle.ekf_ok}")
print(f" Last Heartbeat: {vehicle.last_heartbeat}")
print(f" Is Armable?: {vehicle.is_armable}")
print(f" System status: {vehicle.system_status.state}")
print(f" Mode: {vehicle.mode.name}")
vehicle.close()


SIMPLE TAKE OFF AND RTL WITH STUCK FRAME

In [ ]:
from dronekit import connect, VehicleMode, LocationGlobalRelative
from cvzone.HandTrackingModule import HandDetector
from cvzone.FPS import FPS 
import cv2
import time

# -------------------- DroneKit Setup --------------------
print("[INFO] Connecting to vehicle...")
vehicle = connect('127.0.0.1:14550', wait_ready=True)

def change_mode(mode_name):
    print(f"[INFO] Changing mode to {mode_name}...")
    vehicle.mode = VehicleMode(mode_name)
    while vehicle.mode.name != mode_name:
        print(f"[INFO] Waiting for mode change to {mode_name}...")
        time.sleep(1)
    print(f"[INFO] Mode changed to {vehicle.mode.name}")

def arm_and_takeoff(target_altitude):
    change_mode("GUIDED")

    print("[INFO] Arming motors...")
    vehicle.armed = True
    while not vehicle.armed:
        print("[INFO] Waiting for arming...")
        time.sleep(1)

    print(f"[INFO] Taking off to {target_altitude} meters...")
    vehicle.simple_takeoff(target_altitude)

    while True:
        current_alt = vehicle.location.global_relative_frame.alt
        print(f"[INFO] Current Altitude: {current_alt:.2f} meters")
        if current_alt >= target_altitude * 0.95:
            print("[INFO] Target altitude reached")
            break
        time.sleep(1)

# -------------------- Hand Detection Setup --------------------
cap = cv2.VideoCapture(0)
fpsReader = FPS(avgCount=30)
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.6, minTrackCon=0.5)

# Flags to avoid repeated triggering

has_taken_off = False
rtl_triggered = False

print("[INFO] Starting gesture detection...")
while True:
    success, img = cap.read()
    hands, img = detector.findHands(img, draw=True, flipType=True)

    if hands:
        hand1 = hands[0]
        handType1 = hand1["type"]
        fingers1 = detector.fingersUp(hand1)

        #print(f"[INFO] Right hand fingers up: {fingers1} => Count: {fingers1.count(1)}")

        # 🛫 Takeoff if right thumb only is up
        if handType1 == 'Right' and fingers1[0] and fingers1.count(1) == 1 and not has_taken_off:
            print("[GESTURE] Right thumb detected — initiating TAKEOFF...")
            arm_and_takeoff(12)
            has_taken_off = True
            rtl_triggered = False  # Reset RTL flag just in case

        # 🏁 RTL if all five fingers are up
        elif handType1 == 'Right' and fingers1.count(1) == 5 and has_taken_off and not rtl_triggered:
            print("[GESTURE] All five fingers detected — initiating RTL...")
            change_mode("RTL")
            rtl_triggered = True

    # Display video feed
    cv2.imshow("Drone Gesture Control", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
print("[INFO] Closing connection and camera...")
cap.release()
cv2.destroyAllWindows()
vehicle.close()


TAKE OFF AND RTL WITH VISUALS

In [11]:
from dronekit import connect, VehicleMode, LocationGlobalRelative
from cvzone.HandTrackingModule import HandDetector
from cvzone import putTextRect
from cvzone.FPS import FPS 
import cv2
import time

# -------------------- DroneKit Setup --------------------
print("[INFO] Connecting to vehicle...")
vehicle = connect('127.0.0.1:14550', wait_ready=True)

def change_mode(mode_name):
    print(f"[INFO] Changing mode to {mode_name}...")
    vehicle.mode = VehicleMode(mode_name)
    while vehicle.mode.name != mode_name:
        print(f"[INFO] Waiting for mode change to {mode_name}...")
        time.sleep(0.5)
    print(f"[INFO] Mode changed to {vehicle.mode.name}")

# -------------------- Hand Detection Setup --------------------
cap = cv2.VideoCapture(0)
fpsReader = FPS(avgCount=30)
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.6, minTrackCon=0.5)

# Flags and state tracking
takeoff_requested = False
taking_off = False
rtl_triggered = False
target_altitude = 12
start_takeoff_time = None

print("[INFO] Starting gesture detection...")
while True:
    # Capture and show webcam frame
    success, img = cap.read()
    H,W,_ = img.shape
    hands, img = detector.findHands(img, draw=True, flipType=True)

    # Read current altitude for feedback
    current_alt = vehicle.location.global_relative_frame.alt

    if hands:
        hand1 = hands[0]
        handType1 = hand1["type"]
        fingers1 = detector.fingersUp(hand1)

        #print(f"[INFO] Right hand fingers up: {fingers1} => Count: {fingers1.count(1)}")

        # 🛫 Takeoff Gesture — only right thumb up
        if handType1 == 'Right' and fingers1[4] and fingers1.count(1) == 1:
            if not takeoff_requested and not taking_off:
                print("[GESTURE] Right thumb detected — TAKEOFF initiated")
                change_mode("GUIDED")
                vehicle.armed = True
                takeoff_requested = True
                start_takeoff_time = time.time()

        # 🏁 RTL Gesture — all five fingers up
        if handType1 == 'Right' and fingers1.count(1) == 5:
            if not rtl_triggered and  not taking_off and current_alt  > 1.5:  # Ensure we're in air
                print("[GESTURE] All five fingers detected — initiating RTL")
                change_mode("RTL")
                rtl_triggered = True
                takeoff_requested = False
                taking_off = False

    # ⏳ Handle Takeoff Progress
    if takeoff_requested:
        if vehicle.armed:
            print("[INFO] Motors armed — taking off now...")
            vehicle.simple_takeoff(target_altitude)
            taking_off = True
            takeoff_requested = False

    if taking_off:
        current_alt = vehicle.location.global_relative_frame.alt
        print(f"[INFO] Altitude: {current_alt:.2f} m")
        if current_alt >= target_altitude * 0.95:
            print("[INFO] Target altitude reached ✅")
            taking_off = False
            rtl_triggered = False

    putTextRect(img, text=f'ALTITUDE:{current_alt}m', pos=(int(W*0.34), 450), scale=1.3,  #W-W*0.3 , 50, 100 
                        thickness=2, colorR=(0,0,210), offset=8, border=3, colorB=(0,0,0) )
                        
    putTextRect(img, text=f'MODE: {vehicle.mode.name}', pos=(int(W*0.34), 400), scale=1.3, 
                        thickness=2, colorR=(51,51,0), offset=8, border=3, colorB=(0,0,0) )
    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(10, 470),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    # Show camera feedq
    cv2.imshow("Drone Gesture Control", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
print("[INFO] Closing connection and camera...")
cap.release()
cv2.destroyAllWindows()
vehicle.close()


[INFO] Connecting to vehicle...


CRITICAL:autopilot:APM:Copter V3.3 (d6053245)
CRITICAL:autopilot:Frame: QUAD


[INFO] Starting gesture detection...
[GESTURE] Right thumb detected — TAKEOFF initiated
[INFO] Changing mode to GUIDED...
[INFO] Waiting for mode change to GUIDED...
[INFO] Waiting for mode change to GUIDED...


ERROR:autopilot:ARMING MOTORS
CRITICAL:autopilot:Initialising APM...


[INFO] Mode changed to GUIDED
[INFO] Motors armed — taking off now...
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0

...

[GESTURE] Right thumb detected — TAKEOFF initiated
[INFO] Changing mode to GUIDED...
[INFO] Waiting for mode change to GUIDED...


ERROR:autopilot:ARMING MOTORS
CRITICAL:autopilot:Initialising APM...


[INFO] Mode changed to GUIDED
[INFO] Motors armed — taking off now...
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0.00 m
[INFO] Altitude: 0

ERROR:autopilot:DISARMING MOTORS


[INFO] Closing connection and camera...


ADVANCE MOVEMENTS USING GESTURE

In [15]:
from dronekit import connect, VehicleMode, LocationGlobalRelative
from cvzone.HandTrackingModule import HandDetector
from cvzone import putTextRect
from cvzone.FPS import FPS 
import cv2
import time
import math 

# -------------------- DroneKit Setup --------------------
print("[INFO] Connecting to vehicle...")
vehicle = connect('127.0.0.1:14550', wait_ready=True)

# ---------------- Helper Functions ----------------
def get_distance_meters(loc1, loc2):
    """Calculate approximate ground distance between two LocationGlobalRelative points."""
    dlat = loc2.lat - loc1.lat
    dlon = loc2.lon - loc1.lon
    dalt = loc2.alt - loc1.alt
    return math.sqrt((dlat * 111139.0)**2 + (dlon * 111139.0 * math.cos(math.radians(loc1.lat)))**2 + (dalt)**2)

def goto_relative(north=0, east=0, down=0):
    """Move drone relative to its current position and update global target."""
    global moving, target_location

    current = vehicle.location.global_relative_frame
    new_lat = current.lat + (north / 111139.0)
    new_lon = current.lon + (east / (111139.0 * math.cos(math.radians(current.lat))))
    new_alt = current.alt - down  # Positive down = lower altitude

    target_location = LocationGlobalRelative(new_lat, new_lon, new_alt)
    vehicle.simple_goto(target_location)
    moving = True
    print(f"[INFO] Command sent: N:{north} E:{east} ALT:{new_alt:.2f}")

def change_mode(mode_name):
    print(f"[INFO] Changing mode to {mode_name}...")
    vehicle.mode = VehicleMode(mode_name)
    while vehicle.mode.name != mode_name:
        print(f"[INFO] Waiting for mode change to {mode_name}...")
        time.sleep(0.5)
    print(f"[INFO] Mode changed to {vehicle.mode.name}")

# -------------------- Hand Detection Setup --------------------
cap = cv2.VideoCapture(0)
fpsReader = FPS(avgCount=30)
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.6, minTrackCon=0.5)

# Flags and state tracking
takeoff_requested = False
taking_off = False
rtl_triggered = False
in_air = False 
target_altitude = 12
start_takeoff_time = None

# Movement step in meters
STEP = 10.0
# Threshold to check if vehicle has reached target (in meters)
DISTANCE_THRESHOLD = 1.0
# Flag to indicate whether the drone is already executing a movement
moving = False
target_location = None  # Will store the latest target LocationGlobalRelative

print("[INFO] Starting gesture detection...")
while True:
    # Capture and show webcam frame
    success, img = cap.read()
    H,W,_ = img.shape
    hands, img = detector.findHands(img, draw=True, flipType=True)

    # Read current altitude for feedback
    current_alt = vehicle.location.global_relative_frame.alt

      # Check if movement completed ----------------
    if moving and target_location is not None:
        current = vehicle.location.global_relative_frame
        dist = get_distance_meters(current, target_location)
        putTextRect(img, text=f'MOVING:{dist:.2f}m AWAY', pos=(int(W*0.34), 350), scale=1.3,  #W-W*0.3 , 50, 100 
                        thickness=2, colorR=(0,0,210), offset=8, border=3, colorB=(0,0,0) )
        if dist < DISTANCE_THRESHOLD:
            print("[INFO] Reached target location.")
            moving = False  # Allow next command

    if hands:
        hand1 = hands[0]
        handType1 = hand1["type"]
        fingers1 = detector.fingersUp(hand1)

        #print(f"[INFO] Right hand fingers up: {fingers1} => Count: {fingers1.count(1)}")

        # 🛫 Takeoff Gesture — only right thumb up
        if handType1 == 'Right' and fingers1[0] and fingers1.count(1) == 1 and not vehicle.armed:
            if not takeoff_requested and not taking_off:
                print("[GESTURE] Right thumb detected — TAKEOFF initiated")
                change_mode("GUIDED")
                vehicle.armed = True
                takeoff_requested = True
                start_takeoff_time = time.time()

        # 🏁 RTL Gesture — all five fingers up
        if handType1 == 'Right' and fingers1.count(1) == 5:
            if not rtl_triggered and  not taking_off and current_alt  > 1.5:  # Ensure we're in air
                print("[GESTURE] All five fingers detected — initiating RTL")
                change_mode("RTL")
                rtl_triggered = True
                takeoff_requested = False
                taking_off = False
                in_air = False
                moving = False 
                
        if in_air and not moving:
            # Right Hand: Increase Altitude
            if handType1 == "Right" and fingers1[1] and fingers1[2] and fingers1.count(1) == 2:
                print("[CMD] Increase Altitude +5m")
                goto_relative(down=-STEP)

            # Right Hand: Decrease Altitude
            elif handType1 == "Right" and fingers1[1] and fingers1[4] and fingers1.count(1) == 2:
                print("[CMD] Decrease Altitude -5m")
                goto_relative(down=STEP)

            # Right Hand: Move Right
            elif handType1 == "Right" and fingers1[0] and fingers1[1] and fingers1.count(1) == 2:
                print("[CMD] Move Right +5m")
                goto_relative(east=STEP)

            # Left Hand: Move Left
            elif handType1 == "Left" and fingers1[0] and fingers1[1] and fingers1.count(1) == 2:
                print("[CMD] Move Left -5m")
                goto_relative(east=-STEP)

            # Move Forward (+5m North)
            elif handType1 == "Left" and fingers1[1] and fingers1[2] and fingers1.count(1) == 2:
                print("[CMD] Move Forward +5m")
                goto_relative(north=STEP)

            # Move Backward (-5m North)
            elif handType1 == "Left" and fingers1[1] and fingers1[4] and fingers1.count(1) == 2:
                print("[CMD] Move Backward -5m")
                goto_relative(north=-STEP)

    # ⏳ Handle Takeoff Progress
    if takeoff_requested:
        if vehicle.armed:
            print("[INFO] Motors armed — taking off now...")
            vehicle.simple_takeoff(target_altitude)
            taking_off = True
            takeoff_requested = False

    if taking_off:
        current_alt = vehicle.location.global_relative_frame.alt
        #print(f"[INFO] Altitude: {current_alt:.2f} m")
        if current_alt >= target_altitude * 0.95:
            print("[INFO] Target altitude reached ✅")
            taking_off = False
            rtl_triggered = False
            in_air = True 

    putTextRect(img, text=f'ALTITUDE:{current_alt}m', pos=(int(W*0.34), 450), scale=1.3,  #W-W*0.3 , 50, 100 
                        thickness=2, colorR=(0,0,210), offset=8, border=3, colorB=(0,0,0) )
                        
    putTextRect(img, text=f'MODE: {vehicle.mode.name}', pos=(int(W*0.34), 400), scale=1.3, 
                        thickness=2, colorR=(51,51,0), offset=8, border=3, colorB=(0,0,0) )
    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(10, 470),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    # Show camera feedq
    cv2.imshow("Drone Gesture Control", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
print("[INFO] Closing connection and camera...")
cap.release()
cv2.destroyAllWindows()
vehicle.close()


CRITICAL:autopilot:APM:Copter V3.3 (d6053245)
CRITICAL:autopilot:Frame: QUAD


[INFO] Connecting to vehicle...
[INFO] Starting gesture detection...
[GESTURE] Right thumb detected — TAKEOFF initiated
[INFO] Changing mode to GUIDED...
[INFO] Waiting for mode change to GUIDED...
[INFO] Waiting for mode change to GUIDED...


ERROR:autopilot:ARMING MOTORS
CRITICAL:autopilot:Initialising APM...


[INFO] Mode changed to GUIDED
[INFO] Motors armed — taking off now...
[INFO] Target altitude reached ✅
[CMD] Increase Altitude +5m
[INFO] Command sent: N:0 E:0 ALT:21.99
[INFO] Reached target location.
[CMD] Decrease Altitude -5m
[INFO] Command sent: N:0 E:0 ALT:11.92
[INFO] Reached target location.
[CMD] Move Forward +5m
[INFO] Command sent: N:10.0 E:0 ALT:11.92
[INFO] Reached target location.
[CMD] Move Forward +5m
[INFO] Command sent: N:10.0 E:0 ALT:11.95
[INFO] Reached target location.
[CMD] Move Right +5m
[INFO] Command sent: N:0 E:10.0 ALT:11.94
[INFO] Reached target location.
[CMD] Move Forward +5m
[INFO] Command sent: N:10.0 E:0 ALT:11.92
[INFO] Reached target location.
[CMD] Move Left -5m
[INFO] Command sent: N:0 E:-10.0 ALT:11.89
[INFO] Reached target location.
[CMD] Move Left -5m
[INFO] Command sent: N:0 E:-10.0 ALT:11.88
[INFO] Reached target location.
[CMD] Move Backward -5m
[INFO] Command sent: N:-10.0 E:0 ALT:11.90
[INFO] Reached target location.
[CMD] Move Right +5m
[IN

ERROR:autopilot:DISARMING MOTORS


[INFO] Closing connection and camera...


VIRTUAL DYNAMIC BUTTON

In [4]:
from cvzone.HandTrackingModule import HandDetector
from cvzone import putTextRect
from cvzone.FPS import FPS 
import cv2
import numpy as np 

# Initialize the webcam to capture video
# The '2' indicates the third camera connected to your computer; '0' would usually refer to the built-in camera
cap = cv2.VideoCapture(0)
fpsReader = FPS(avgCount=30)

# Initialize the HandDetector class with the given parameters
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.5, minTrackCon=0.5)
cxrect,cyrect,wrect, hrect = 200,200,100,100
alpha=0.5
movement= False 
colorR = 55,55,0


# Continuously get frames from the webcam
while True:
    # Capture each frame from the webcam
    # 'success' will be True if the frame is successfully captured, 'img' will contain the frame
    success, img = cap.read()
    H,W,_ = img.shape  #H,W = 480, 640
    #print(H,W)

    # Configure the virtual zone

    x1zone, y1zone, x2zone, y2zone = int(W*0.2), int(H*0.2), int(W*0.8), int(H*0.8) 
    wzone = int(x2zone-x1zone)
    hzone = int(y2zone-y1zone)
    cxzone, cyzone = int((x1zone+x2zone)//2), int((y1zone+y2zone)//2)
    cv2.circle(img, (cxzone,cyzone), 6, (255,0,0), cv2.FILLED)


    cv2.rectangle(img, (x1zone,y1zone), (x2zone,y2zone), (25,0,51),3)
    #cv2.rectangle(img, (int(200),int(200)), (int(220),int(220)), (0,0,0),cv2.FILLED)

    # Find hands in the current frame
    # The 'draw' parameter draws landmarks and hand outlines on the image if set to True
    # The 'flipType' parameter flips the image, making it easier for some detections
    hands, img = detector.findHands(img, draw=True, flipType=True)

    # Check if any hands are detected
    if hands:
        # Information for the first hand detected
        hand1 = hands[0]  # Get the first hand detected
        lmList1 = hand1["lmList"]  # List of 21 landmarks for the first hand
        bbox1 = hand1["bbox"]  # Bounding box around the first hand (x,y,w,h coordinates)
        center1 = hand1['center']  # Center coordinates of the first hand
        handType1 = hand1["type"]  # Type of the first hand ("Left" or "Right")
        length, info, img = detector.findDistance(lmList1[8][0:2], lmList1[12][0:2], img, color=(0, 0, 255),scale=10) #Thumb and later
        
        cx1, cy1 = center1
        cv2.line(img, (cx1,0),(cx1,H), (0,0,255),4)
        cv2.line(img, (0,cy1),(W,cy1), (0,0,255),4)
        cv2.circle(img, (cx1,cy1), 6, (0,255,0), cv2.FILLED)

        if cx1>int(W*0.2) and cx1<int(W*0.8) and cy1>int(H*0.2) and cy1< int(H*0.8): 

            if length<40: 
                if cx1>int(cxrect-wrect*0.5) and cx1<int(cxrect+wrect*0.5) and cy1>int(cyrect-hrect*0.5) and cy1< int(cyrect+hrect*0.5):
                    colorR = 0,255,0
                    cxrect, cyrect = cx1, cy1

                    dx_pixels = cx1 - cxzone
                    dy_pixels = cyzone - cy1  # Invert Y

                    east_offset = np.interp(dx_pixels, [-wzone*0.5, wzone*0.5], [-80, 80])
                    north_offset = np.interp(dy_pixels, [-hzone*0.5, hzone*0.5], [-80, 80])
                    putTextRect(img, text=f'D: {east_offset:0.2f},{north_offset:0.2f}', pos=(int(W*0.34), 400), scale=1.3, 
                        thickness=2, colorR=(51,51,0), offset=8, border=3, colorB=(0,0,0) )
                    movement = True 
                else: 
                    colorR = 55,55,0
            else: 
                    colorR = 55,55,0
                    movement = False
                    
        
        #cv2.rectangle(img, (int(cxrect-wrect*0.5),int(cyrect-hrect*0.5)), (int(cxrect+wrect*0.5),int(cyrect+hrect*0.5)), colorR, cv2.FILLED)
        
        # Create a copy of the image to draw the overlay
        overlay = img.copy()
        # Draw the filled rectangle on the overlay image
        cv2.rectangle(overlay,
                    (int(cxrect - wrect * 0.5), int(cyrect - hrect * 0.5)),
                    (int(cxrect + wrect * 0.5), int(cyrect + hrect * 0.5)),
                    colorR, thickness=cv2.FILLED)

        # Blend the overlay with the original image
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)



            # Count the number of fingers up for the first hand
            #fingers1 = detector.fingersUp(hand1)
            #print(fingers1)

            #print(f'H1 = {fingers1.count(1)}', end=" ")  # Print the count of fingers that are up


        # Check if a second hand is detected
        if len(hands) == 2:
            # Information for the second hand
            hand2 = hands[1]
            lmList2 = hand2["lmList"]
            bbox2 = hand2["bbox"]
            center2 = hand2['center']
            handType2 = hand2["type"]

            #Count the number of fingers up for the second hand
            fingers2 = detector.fingersUp(hand2)
        #     print(f'H2 = {fingers2.count(1)}', end=" ")



    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(10, 470),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    # Display the image in a window
    cv2.imshow("Drone Simulation", img)

    # Keep the window open and update it for each frame; wait for 1 millisecond between frames
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


MAPPED POSITIONAL GESTURE BASED CONTROL

In [9]:
from dronekit import connect, VehicleMode, LocationGlobalRelative
from cvzone.HandTrackingModule import HandDetector
from cvzone import putTextRect, overlayPNG
from cvzone.FPS import FPS 
import cv2
import time
import math 
import numpy as np

# -------------------- DroneKit Setup --------------------
print("[INFO] Connecting to vehicle...")
vehicle = connect('127.0.0.1:14550', wait_ready=True)

#Home Parameters
lat_home, lon_home = vehicle.location.global_relative_frame.lat, vehicle.location.global_relative_frame.lon

# ---------------- Helper Functions ----------------
def get_distance_meters(loc1, loc2):
    """Calculate approximate ground distance between two LocationGlobalRelative points."""
    dlat = loc2.lat - loc1.lat
    dlon = loc2.lon - loc1.lon
    dalt = loc2.alt - loc1.alt
    return math.sqrt((dlat * 111139.0)**2 + (dlon * 111139.0 * math.cos(math.radians(loc1.lat)))**2 + (dalt)**2)

def goto_relative(north=0, east=0, down=0):
    """Move drone relative to its current position and update global target."""
    global moving, target_location

    current = vehicle.location.global_relative_frame
    new_lat = lat_home + (north / 111139.0)
    new_lon = lon_home + (east / (111139.0 * math.cos(math.radians(current.lat))))
    new_alt = current.alt - down  # Positive down = lower altitude

    target_location = LocationGlobalRelative(new_lat, new_lon, new_alt)
    vehicle.simple_goto(target_location)
    moving = True
    print(f"[INFO] Command sent: N:{north} E:{east} ALT:{new_alt:.2f}")

def change_mode(mode_name):
    print(f"[INFO] Changing mode to {mode_name}...")
    vehicle.mode = VehicleMode(mode_name)
    while vehicle.mode.name != mode_name:
        print(f"[INFO] Waiting for mode change to {mode_name}...")
        time.sleep(0.5)
    print(f"[INFO] Mode changed to {vehicle.mode.name}")

# -------------------- Hand Detection Setup --------------------
cap = cv2.VideoCapture(0)
imgDrone = cv2.imread(r'C:\Users\hoque\Desktop\OpenCV\DroneKit\DronePng.png', cv2.IMREAD_UNCHANGED)
imgDrone = cv2.resize(imgDrone, (60,60))
fpsReader = FPS(avgCount=30)
detector = HandDetector(staticMode=False, maxHands=1, modelComplexity=1, detectionCon=0.6, minTrackCon=0.5)
cxrect,cyrect,wrect, hrect = 320,240,100,100
alpha=0.25
colorR=55,55,0

# Flags and state tracking
takeoff_requested = False
taking_off = False
rtl_triggered = False
in_air = False 
start_takeoff_time = None
movement= False 
# Flag to indicate whether the drone is already executing a movement
moving = False
target_location = None  # Will store the latest target LocationGlobalRelative

# Movement step in meters
STEP = 10.0
# Threshold to check if vehicle has reached target (in meters)
DISTANCE_THRESHOLD = 1.0
target_altitude = 12


print("[INFO] Starting gesture detection...")
while True:
    # Capture and show webcam frame
    success, img = cap.read()
    H,W,_ = img.shape
    x1zone, y1zone, x2zone, y2zone = int(W*0.2), int(H*0.2), int(W*0.8), int(H*0.8) #Coordinates if the virtual zone
    wzone = int(x2zone-x1zone)
    hzone = int(y2zone-y1zone)
    cxzone, cyzone = int((x1zone+x2zone)//2), int((y1zone+y2zone)//2)
    cv2.circle(img, (cxzone,cyzone), 6, (255,0,0), cv2.FILLED)


    cv2.rectangle(img, (x1zone,y1zone), (x2zone,y2zone), (25,0,51),3)
    hands, img = detector.findHands(img, draw=True, flipType=True)

    # Read current altitude for feedback
    current_alt = vehicle.location.global_relative_frame.alt

      # Check if movement completed ----------------
    if moving and target_location is not None:
        current = vehicle.location.global_relative_frame
        dist = get_distance_meters(current, target_location)
        putTextRect(img, text=f'MOVING:{dist:.2f}m AWAY', pos=(int(W*0.34), 350), scale=1.3,  #W-W*0.3 , 50, 100 
                        thickness=2, colorR=(0,0,210), offset=8, border=3, colorB=(0,0,0) )
        if dist < DISTANCE_THRESHOLD:
            print("[INFO] Reached target location.")
            moving = False  # Allow next command

    if hands:
       # Information for the first hand detected
        hand1 = hands[0]  # Get the first hand detected
        lmList1 = hand1["lmList"]  # List of 21 landmarks for the first hand
        bbox1 = hand1["bbox"]  # Bounding box around the first hand (x,y,w,h coordinates)
        center1 = hand1['center']  # Center coordinates of the first hand
        handType1 = hand1["type"]  # Type of the first hand ("Left" or "Right")
        fingers1 = detector.fingersUp(hand1)
        
        
        #cx1, cy1 = center1
        cx1, cy1 = lmList1[12][0:2] #8
        #print(f"[INFO] Right hand fingers up: {fingers1} => Count: {fingers1.count(1)}")

        # 🛫 Takeoff Gesture — only right thumb up
        if handType1 == 'Left' and fingers1[0] and fingers1.count(1) == 1 and not vehicle.armed:
            if not takeoff_requested and not taking_off:
                print("[GESTURE] Right thumb detected — TAKEOFF initiated")
                change_mode("GUIDED")
                vehicle.armed = True
                takeoff_requested = True
                start_takeoff_time = time.time()

        # 🏁 RTL Gesture — all five fingers up
        if handType1 == 'Left' and fingers1.count(1) == 5:
            if not rtl_triggered and  not taking_off and current_alt  > 1.5:  # Ensure we're in air
                print("[GESTURE] All five fingers detected — initiating RTL")
                change_mode("RTL")
                rtl_triggered = True
                takeoff_requested = False
                taking_off = False
                in_air = False
                moving = False 
                
        if in_air and not moving:
            # Right Hand: Increase Altitude
            if handType1 == "Left" and fingers1[1] and fingers1[2] and fingers1.count(1) == 2:
                print("[CMD] Increase Altitude +5m")
                goto_relative(down=-STEP)

            # Right Hand: Decrease Altitude
            elif handType1 == "Left" and fingers1[1] and fingers1[4] and fingers1.count(1) == 2:
                print("[CMD] Decrease Altitude -5m")
                goto_relative(down=STEP)
            
            # Positional Logic
            if handType1 == 'Right' and cx1>int(W*0.2) and cx1<int(W*0.8) and cy1>int(H*0.2) and cy1< int(H*0.8)\
                                                                                 and not rtl_triggered: 
               
                length, info, img = detector.findDistance(lmList1[8][0:2], lmList1[12][0:2], img, color=(0, 0, 255),scale=10) #Thumb and later
                #Draw the coordinates line
                cv2.line(img, (cx1,0),(cx1,H), (0,0,255),3)
                cv2.line(img, (0,cy1),(W,cy1), (0,0,255),3)
                cv2.circle(img, (cx1,cy1), 6, (0,255,0), cv2.FILLED)
                
                if length<40: 
                    if cx1>int(cxrect-wrect*0.5) and cx1<int(cxrect+wrect*0.5) and cy1>int(cyrect-hrect*0.5) and cy1< int(cyrect+hrect*0.5):
                        colorR = 0,255,0
                        cxrect, cyrect = cx1, cy1

                        dx_pixels = cx1 - cxzone
                        dy_pixels = cyzone - cy1  # Invert Y

                        east_offset = np.interp(dx_pixels, [-wzone*0.5, wzone*0.5], [-80, 80])
                        north_offset = np.interp(dy_pixels, [-hzone*0.5, hzone*0.5], [-80, 80])
                        putTextRect(img, text=f'D: {east_offset:0.2f},{north_offset:0.2f}', pos=(int(cx1)+40, int(cy1)+40), scale=1.1, 
                            thickness=2, colorR=(51,51,0), offset=7, border=3, colorB=(0,0,0) )
                        movement = True 
                    else: 
                        colorR = 55,55,0
                else:
                     if movement: 
                            print(f"[CMD] Moving to target{east_offset:0.2f},{north_offset:0.2f}m")
                            goto_relative(north=north_offset, east=east_offset)
                            movement = False
                            colorR = 55,55,0
                
                                
            else: 
                colorR = 55,55,0
                
                
    # Create a copy of the image to draw the overlay
    overlay = img.copy()
    # Draw the filled rectangle on the overlay image
    cv2.rectangle(overlay,
                (int(cxrect - wrect * 0.5), int(cyrect - hrect * 0.5)),
                (int(cxrect + wrect * 0.5), int(cyrect + hrect * 0.5)),
                colorR, thickness=cv2.FILLED)
    
    # Blend the overlay with the original image
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
    overlayPNG(img, imgDrone, [int(cxrect-30), int(cyrect-30)])

    

    # ⏳ Handle Takeoff Progress
    if takeoff_requested:
        if vehicle.armed:
            print("[INFO] Motors armed — taking off now...")
            vehicle.simple_takeoff(target_altitude)
            taking_off = True
            takeoff_requested = False

    if taking_off:
        current_alt = vehicle.location.global_relative_frame.alt
        #print(f"[INFO] Altitude: {current_alt:.2f} m")
        if current_alt >= target_altitude * 0.95:
            print("[INFO] Target altitude reached ✅")
            taking_off = False
            rtl_triggered = False
            in_air = True 
            moving = False 

    putTextRect(img, text=f'MODE: {vehicle.mode.name}', pos=(int(W*0.34), 420), scale=1.3, 
                        thickness=2, colorR=(51,51,0), offset=8, border=3, colorB=(0,0,0) )
    putTextRect(img, text=f'ALTITUDE:{current_alt}m', pos=(int(W*0.34), 460), scale=1.3,  #W-W*0.3 , 50, 100 
                        thickness=2, colorR=(0,0,210), offset=8, border=3, colorB=(0,0,0) )
                        
   
    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(10, 470),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)
    # Show camera feedq
    cv2.imshow("Drone Gesture Control", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanupdronekit-sitl copter --home=23.4613785,91.1767227,10,270
print("[INFO] Closing connection and camera...")
cap.release()
cv2.destroyAllWindows()
vehicle.close()


[INFO] Connecting to vehicle...


CRITICAL:autopilot:APM:Copter V3.3 (d6053245)
CRITICAL:autopilot:Frame: QUAD


[INFO] Starting gesture detection...
[GESTURE] Right thumb detected — TAKEOFF initiated
[INFO] Changing mode to GUIDED...
[INFO] Waiting for mode change to GUIDED...
[INFO] Waiting for mode change to GUIDED...
[INFO] Mode changed to GUIDED


ERROR:autopilot:ARMING MOTORS
CRITICAL:autopilot:GROUND START
CRITICAL:autopilot:Initialising APM...


[INFO] Motors armed — taking off now...
[INFO] Target altitude reached ✅
[CMD] Increase Altitude +5m
[INFO] Command sent: N:0 E:0 ALT:21.98
[INFO] Reached target location.
[CMD] Decrease Altitude -5m
[INFO] Command sent: N:0 E:0 ALT:11.96
[INFO] Reached target location.
[CMD] Moving to target-49.17,50.56m
[INFO] Command sent: N:50.55555555555557 E:-49.166666666666664 ALT:11.98
[INFO] Reached target location.
[CMD] Moving to target24.17,-14.44m
[INFO] Command sent: N:-14.444444444444443 E:24.16666666666667 ALT:11.96
[INFO] Reached target location.
[CMD] Moving to target-19.17,60.00m
[INFO] Command sent: N:60.0 E:-19.166666666666664 ALT:11.95
[INFO] Reached target location.
[CMD] Moving to target-2.08,5.00m
[INFO] Command sent: N:5.0 E:-2.0833333333333286 ALT:11.94
[INFO] Reached target location.
[CMD] Increase Altitude +5m
[INFO] Command sent: N:0 E:0 ALT:21.92
[INFO] Reached target location.
[CMD] Moving to target-49.58,-28.89m
[INFO] Command sent: N:-28.888888888888886 E:-49.583333333

ERROR:autopilot:DISARMING MOTORS


[INFO] Closing connection and camera...


In [4]:
vehicle.close()